# How to create a ReAct agent from scratch

In [41]:
from typing import (
    Annotated,
    Sequence,
    TypedDict,
)
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages


class AgentState(TypedDict):
    """The state of the agent."""

    # add_messages is a reducer
    # See https://langchain-ai.github.io/langgraph/concepts/low_level/#reducers
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [42]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

model = ChatOpenAI(model="gpt-4o-mini")


@tool
def get_weather(location: str):
    """Call to get the weather from a specific location."""
    # This is a placeholder for the actual implementation
    # Don't let the LLM know this though 😊
    if any([city in location.lower() for city in ["sf", "san francisco"]]):
        return "It's sunny in San Francisco, but you better look out if you're a Gemini 😈."
    else:
        return f"I am not sure what the weather is in {location}"


tools = [get_weather]

model = model.bind_tools(tools)

In [1]:
import json
from langchain_core.messages import ToolMessage, SystemMessage
from langchain_core.runnables import RunnableConfig

tools_by_name = {tool.name: tool for tool in tools}


# Define our tool node
def tool_node(state: AgentState):
    outputs = []
    for tool_call in state["messages"][-1].tool_calls:
        tool_result = tools_by_name[tool_call["name"]].invoke(tool_call["args"])
        outputs.append(
            ToolMessage(
                content=json.dumps(tool_result),
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )
    return {"messages": outputs}


# Define the node that calls the model
def call_model(
    state: AgentState,
    config: RunnableConfig,
):
    # this is similar to customizing the create_react_agent with state_modifier, but is a lot more flexible
    system_prompt = SystemMessage(
        "You are a helpful AI assistant, please respond to the users query to the best of your ability!"
    )
    response = model.invoke([system_prompt] + state["messages"], config)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# Define the conditional edge that determines whether to continue or not
def should_continue(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"

NameError: name 'tools' is not defined

In [44]:
from langgraph.graph import StateGraph, END

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "tools",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("tools", "agent")

# Now we can compile and visualize our graph
graph = workflow.compile(checkpointer=)

from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

SyntaxError: expected argument value expression (3624691405.py, line 40)

In [6]:
# Helper function for formatting the stream nicely
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()


inputs = {"messages": [("user", "what is the weather in sf")]}
print_stream(graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================

what is the weather in sf
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_qcAL2hUlw8dGxK3OWQJIdc8a)
 Call ID: call_qcAL2hUlw8dGxK3OWQJIdc8a
  Args:
    location: San Francisco
================================= Tool Message =================================
Name: get_weather

"It's sunny in San Francisco, but you better look out if you're a Gemini \ud83d\ude08."
================================== Ai Message ==================================

The weather in San Francisco is sunny right now. Enjoy the nice day! However, if you're a Gemini, you might want to keep a lookout for any surprises! 😈


# How to use Postgres checkpointer for persistence

In [38]:
from typing import Literal

from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.postgres import PostgresSaver
from langgraph.checkpoint.postgres.aio import AsyncPostgresSaver
from psycopg_pool import ConnectionPool

# 공통 환경 설정
from dotenv import load_dotenv
import os

# 작업디렉토리를 상위경로로 변경
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
os.chdir(parent_dir)

# 환경변수 설정
load_dotenv(dotenv_path=".env.testcase", override=True)


DB_URI = os.getenv("DATABASE_URL")
connection_kwargs = {
    "autocommit": True,
    "prepare_threshold": 0,
}

with ConnectionPool(
    # Example configuration
    conninfo=DB_URI,
    max_size=20,
    kwargs=connection_kwargs,
) as pool:
    checkpointer = PostgresSaver(pool)

    # NOTE: you need to call .setup() the first time you're using your checkpointer
    # checkpointer.setup()

    graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
    config = {"configurable": {"thread_id": "5"}}
    res = graph.invoke({"messages": [("human", "what's the weather in seoul")]}, config)
    checkpoint = checkpointer.get(config)

In [39]:
res

{'messages': [HumanMessage(content="what's the weather in seoul", additional_kwargs={}, response_metadata={}, id='8c280dfd-195d-4fc9-bd13-503ab06e18c6'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jRMl7C0ES0lvfv6u2WV8XZLt', 'function': {'arguments': '{"location":"Seoul"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 54, 'total_tokens': 70, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d742f5cc-95b9-4a99-8323-0693da26c2a6-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Seoul'}, 'id': 'call_jRMl7C0ES0lvfv6u2WV8XZLt', 'type': 'tool_call'}], usage_metadata={'inp

In [40]:
checkpoint

{'v': 1,
 'id': '1efd864d-f807-62b8-8005-84b9d19bc75b',
 'ts': '2025-01-22T02:02:02.984032+00:00',
 'pending_sends': [],
 'versions_seen': {'agent': {'tools': '00000000000000000000000000000006.0.6122398421424992',
   'start:agent': '00000000000000000000000000000002.0.1736179688598226'},
  'tools': {'branch:agent:should_continue:tools': '00000000000000000000000000000005.0.2572518556617016'},
  '__input__': {},
  '__start__': {'__start__': '00000000000000000000000000000001.0.7546723407945038'}},
 'channel_versions': {'agent': '00000000000000000000000000000007.0.27340668507953136',
  'tools': '00000000000000000000000000000007.0.899193287002628',
  'messages': '00000000000000000000000000000007.0.714841672461274',
  '__start__': '00000000000000000000000000000002.0.8468989404996994',
  'start:agent': '00000000000000000000000000000003.0.4520632624538823',
  'branch:agent:should_continue:tools': '00000000000000000000000000000006.0.3845011818464521'},
 'channel_values': {'agent': 'agent',
  'me

# CodeChatAgent Test

In [8]:
# 공통 환경 설정
import uuid
from dotenv import load_dotenv
import os


# 작업디렉토리를 상위경로로 변경
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
os.chdir(parent_dir)

# 환경변수 설정
load_dotenv(dotenv_path=".env.testcase", override=True)


from langchain_core.messages import HumanMessage
from app.chain_graph.code_chat_agent import CodeChatAgent
from psycopg_pool import AsyncConnectionPool
from psycopg_pool import ConnectionPool
from langgraph.checkpoint.postgres.aio import AsyncPostgresSaver
from langgraph.checkpoint.postgres import PostgresSaver

DB_URI = os.getenv("DATABASE_URL")
connection_kwargs = {
    "autocommit": True,
    "prepare_threshold": 0,
}
agent = CodeChatAgent(index_name="cg_code_assist")

with ConnectionPool(
    conninfo=DB_URI,
    max_size=20,
    kwargs=connection_kwargs,
) as pool:
    thread_id = str(uuid.uuid4())
    checkpointer = PostgresSaver(pool)

    graph, _ = agent.get_chain(thread_id=thread_id, checkpointer=checkpointer)
    config = {"configurable": {"thread_id": thread_id}}

    input_message = HumanMessage(content="안녕! 내 이름은 홍길동이야")
    for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
        event["messages"][-1].pretty_print()


    input_message = HumanMessage(content="내 이름이 뭐야?")
    for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
        event["messages"][-1].pretty_print()


### cg_code_assist 인덱스를 메모리에서 로드했습니다.
### Index already exists: cg_code_assist
================================ Human Message =================================

안녕! 내 이름은 홍길동이야
================================== Ai Message ==================================

안녕하세요 홍길동님! 만나서 반갑습니다. 저는 Claude라는 인공지능 assistance입니다. 궁금한 점이나 도움이 필요한 게 있다면 언제든 물어봐주세요. 최선을 다해 답변하고 도와드리겠습니다.
================================ Human Message =================================

내 이름이 뭐야?
================================== Ai Message ==================================

당신의 이름은 홍길동이라고 하셨습니다.
